In [83]:
from ipyleaflet import (Map, GeoData, basemaps, WidgetControl, GeoJSON,
 LayersControl, Icon, Marker,basemap_to_tiles, Choropleth,
 MarkerCluster, Heatmap,SearchControl, DivIcon,
 FullScreenControl)
from ipywidgets import Text, HTML
from branca.colormap import linear
import geopandas as gpd
import json
from datasources.yahoo import YahooResource

In [12]:
# Countries
countries = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

In [13]:
countries

,pop_est,continent,name,iso_a3,gdp_md_est,geometry
0,920938,Oceania,Fiji,FJI,8374.0,"MULTIPOLYGON (((180.00000 -16.06713, 180.00000..."
1,53950935,Africa,Tanzania,TZA,150600.0,"POLYGON ((33.90371 -0.95000, 34.07262 -1.05982..."
2,603253,Africa,W. Sahara,ESH,906.5,"POLYGON ((-8.66559 27.65643, -8.66512 27.58948..."
3,35623680,North America,Canada,CAN,1674000.0,"MULTIPOLYGON (((-122.84000 49.00000, -122.9742..."
4,326625791,North America,United States of America,USA,18560000.0,"MULTIPOLYGON (((-122.84000 49.00000, -120.0000..."
...,...,...,...,...,...,...
172,7111024,Europe,Serbia,SRB,101800.0,"POLYGON ((18.82982 45.90887, 18.82984 45.90888..."
173,642550,Europe,Montenegro,MNE,10610.0,"POLYGON ((20.07070 42.58863, 19.80161 42.50009..."
174,1895250,Europe,Kosovo,-99,18490.0,"POLYGON ((20.59025 41.85541, 20.52295 42.21787..."
175,1218208,North America,Trinidad and Tobago,TTO,43570.0,"POLYGON ((-61.68000 10.76000, -61.10500 10.890..."


In [108]:
from pydantic import BaseModel
from typing import List, Tuple, Any

In [84]:
yahoo = YahooResource()

In [87]:
import datetime as dt

In [144]:
as_of = dt.date.today() - dt.timedelta(days=3)
as_of

datetime.date(2021, 2, 12)

In [196]:
record = yahoo.ohlcav(from_=as_of, to_=as_of, symbol='3067.HK')
100 * float(record.Close / record.Open - 1)

0.46424324678535633

In [167]:
class Index(BaseModel):
    name: str
    symbol: str

In [177]:
class MarketInfo(BaseModel):
    name: str 
    geo: List[float]
    tz: int
    ix1: Index
    ixt: Index = None
                
    def change(self, symbol: str, as_of: dt.date):
        yahoo = YahooResource()
        record = yahoo.ohlcav(from_=as_of, to_=as_of, symbol=symbol)
        return round(100 * float(record.Close / record.Open - 1), 2)
        
    def style_for(self, change: float):
        if change < -.1:
            return 'background-color: #FF8080; color: #FFFFFF'
        elif -.1 < change < .1:
            return 'background-color: #808080; color: #FFFFFF'
        else:
            return 'background-color: #20E0C0; color: #FFFFFF'
    
    def to_html(self, as_of: dt.date):
        ix1_change = self.change(self.ix1.symbol, as_of)
        if self.ixt:
            ixt_change = self.change(self.ixt.symbol, as_of)
        
        return (
            '<span>'+
            f'<div style="">&nbsp;{self.name} GMT {self.tz:+2}</div>' +
            f'<div style="{self.style_for(ix1_change)}">&nbsp;{self.ix1.name}: {ix1_change}</div>' +
            (f'<div style="{self.style_for(ixt_change)}">&nbsp;{self.ixt.name}: {ixt_change}</div>' if self.ixt else "") +
            '</span>'
        ) 


In [181]:
nyse = MarketInfo(geo=[42.9, -78.9], name="NYSE", tz=-5,
                  ix1=Index(name='DOW', symbol='^DJI'),
                  ixt=Index(name='NASDAQ', symbol='^IXIC'))

In [182]:
london = MarketInfo(geo=[51.5, 0.1], name="London", tz=0, 
                    ix1=Index(name='FTSE100', symbol='^FTSE'),
                    ixt=None)

In [197]:
hke = MarketInfo(geo=[22, 114], name="Hong Kong", tz=8, 
                    ix1=Index(name='Hang Seng', symbol='^HSI'),
                    ixt=Index(name='HKD', symbol='3067.HK'))

In [202]:
markets = [london, nyse, hke]
center = [51.5, 0.1]
zoom = 3
m = Map(basemap=basemaps.Esri.WorldImagery, center=center, zoom=zoom)
m = Map(basemap=basemaps.CartoDB.Positron, center=center, zoom=zoom)
m.layout.height="1000px"
for market in markets:
    icon = DivIcon(html=market.to_html(as_of), bg_pos=[0, 0], icon_size=[120, 36+(18 if market.ixt else 0)])
    mark = Marker(location=market.geo, icon=icon)
    m.add_layer(mark);
m

Map(center=[51.5, 0.1], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_…